# NQueens using hill climbing search algorithm


In [11]:
# row-swapping method
# It selects the best neighbor based on the lowest cost found in the neighborhood generated through row swaps.
# It includes a detailed table of possible moves and their costs after each step, showcasing the neighbors generated by the swaps.
# (Generates neighbors by swapping two queens in different columns, which allows for more drastic changes in the state but might skip over good configurations.)
import random
from tabulate import tabulate

# Function to calculate the number of attacking pairs of queens
def calculate_cost(state):
    cost = 0
    n = len(state)
    for i in range(n):
        for j in range(i + 1, n):
            if state[i] == state[j]:  # Same row
                cost += 1
            if abs(state[i] - state[j]) == abs(i - j):  # Same diagonal
                cost += 1
    return cost

# Function to generate neighbors by swapping two columns
def generate_neighbors(state):
    neighbors = []
    n = len(state)
    swaps = []
    for i in range(n):
        for j in range(i + 1, n):
            new_state = state[:]
            new_state[i], new_state[j] = new_state[j], new_state[i]  # Swap rows of queens
            neighbors.append(new_state)
            swaps.append((i + 1, j + 1))  # Record which rows were swapped (1-based indexing)
    return neighbors, swaps

# Function to display the board (matrix) representation using 1-based indexing
def display_board(state):
    n = len(state)
    board = [["."] * n for _ in range(n)]  # Initialize an empty board with dots
    for col in range(n):
        board[state[col] - 1][col] = "Q"  # Place queens in the correct row for each column
    return board

# Hill Climbing Algorithm
def hill_climbing(n, initial_state=None):
    # If no initial state provided, generate a random state
    if initial_state is None:
        current_state = random.sample(range(1, n + 1), n)  # 1-based index for random generation
    else:
        current_state = initial_state

    current_cost = calculate_cost(current_state)
    steps = 0

    print(f"\nInitial State: {current_state}, Cost: {current_cost}")
    print("\nInitial Board Configuration:")
    board = display_board(current_state)
    for row in board:
        print(" ".join(row))

    while current_cost > 0:
        neighbors, swaps = generate_neighbors(current_state)
        next_state = None
        next_cost = current_cost
        swap_pair = None

        # Table to store all neighbors and their costs
        table = []
        for idx, neighbor in enumerate(neighbors):
            neighbor_cost = calculate_cost(neighbor)
            table.append([neighbor, neighbor_cost, swaps[idx]])
            if neighbor_cost < next_cost:
                next_state = neighbor
                next_cost = neighbor_cost
                swap_pair = swaps[idx]  # Save the swap that was selected

        # Print the table of all possible moves
        print("\nPossible moves and their costs (with swapped rows):")
        print(tabulate(table, headers=["Neighbor State", "Cost", "Swapped Rows"]))

        # If no better state is found, we're at a local minimum
        if next_cost >= current_cost:
            print(f"\nLocal minimum reached after {steps} steps with state: {current_state}")
            return current_state, current_cost

        # Move to the best neighbor
        current_state = next_state
        current_cost = next_cost
        steps += 1

        print(f"\nStep {steps}: Selected State: {current_state}, Cost: {current_cost}")
        print(f"Swapped Rows: {swap_pair}")

        # Display the new board configuration after the swap
        print("\nBoard Configuration:")
        board = display_board(current_state)
        for row in board:
            print(" ".join(row))

    print(f"\nGoal state found after {steps} steps with state: {current_state}")
    return current_state, current_cost

# Input number of queens
n = int(input("Enter the number of queens: "))

# Input initial positions of queens (optional) with 1-based index
initial_positions = input(f"Enter the initial positions of {n} queens (e.g., '1 2 3 4'), or leave blank for random: ").strip()
if initial_positions:
    initial_state = list(map(int, initial_positions.split()))
else:
    initial_state = None

# Run the Hill Climbing algorithm
final_state, final_cost = hill_climbing(n, initial_state)

# Output the final result
print(f"\nFinal state: {final_state}")
print(f"Final cost: {final_cost}")


Enter the number of queens: 4
Enter the initial positions of 4 queens (e.g., '1 2 3 4'), or leave blank for random: 4 2 3 1

Initial State: [4, 2, 3, 1], Cost: 2

Initial Board Configuration:
. . . Q
. Q . .
. . Q .
Q . . .

Possible moves and their costs (with swapped rows):
Neighbor State      Cost  Swapped Rows
----------------  ------  --------------
[2, 4, 3, 1]           1  (1, 2)
[3, 2, 4, 1]           1  (1, 3)
[1, 2, 3, 4]           6  (1, 4)
[4, 3, 2, 1]           6  (2, 3)
[4, 1, 3, 2]           1  (2, 4)
[4, 2, 1, 3]           1  (3, 4)

Step 1: Selected State: [2, 4, 3, 1], Cost: 1
Swapped Rows: (1, 2)

Board Configuration:
. . . Q
Q . . .
. . Q .
. Q . .

Possible moves and their costs (with swapped rows):
Neighbor State      Cost  Swapped Rows
----------------  ------  --------------
[4, 2, 3, 1]           2  (1, 2)
[3, 4, 2, 1]           2  (1, 3)
[1, 4, 3, 2]           4  (1, 4)
[2, 3, 4, 1]           4  (2, 3)
[2, 1, 3, 4]           2  (2, 4)
[2, 4, 1, 3]           0 

In [18]:
# class method taught in notes
# Specifically attempts to move each queen to a different row in its column, checking all valid positions and selecting the best move based on the calculated objective (i.e., number of attacking pairs).
from random import randint

# Take input for the size of the board
N = int(input("Enter the size of the board (N): "))

def configureFromInput(board, state):
    print("Enter the initial positions of the queens (0 to N-1 for each column):")
    for i in range(N):
        while True:
            try:
                position = int(input(f"Position for queen in column {i} (row 0 to {N-1}): "))
                if 0 <= position < N:
                    state[i] = position
                    board[position][i] = 1
                    break
                else:
                    print("Invalid position. Please enter a value between 0 and", N-1)
            except ValueError:
                print("Invalid input. Please enter a number.")

def printBoard(board):
    for row in board:
        print(' '.join(str(x) for x in row))

def compareStates(state1, state2):
    return state1 == state2

def fill(board, value):
    for i in range(N):
        for j in range(N):
            board[i][j] = value

def calculateObjective(board, state):
    attacking = 0
    for i in range(N):
        row = state[i]
        for j in range(N):
            if j != i:  # Check other columns
                if board[row][j] == 1:  # Same row
                    attacking += 1
                # Check diagonals
                if row + j - i >= 0 and row + j - i < N and board[row + j - i][j] == 1:
                    attacking += 1
                if row - j + i >= 0 and row - j + i < N and board[row - j + i][j] == 1:
                    attacking += 1
    return attacking // 2

def generateBoard(board, state):
    fill(board, 0)
    for i in range(N):
        board[state[i]][i] = 1

def getNeighbour(board, state):
    opBoard = [[0] * N for _ in range(N)]
    opState = state.copy()
    generateBoard(opBoard, opState)
    opObjective = calculateObjective(opBoard, opState)

    NeighbourBoard = [[0] * N for _ in range(N)]
    NeighbourState = state.copy()
    generateBoard(NeighbourBoard, NeighbourState)

    for i in range(N):
        original_row = NeighbourState[i]
        for j in range(N):
            if j != original_row:  # Avoid the same row
                NeighbourState[i] = j
                NeighbourBoard[j][i] = 1
                NeighbourBoard[original_row][i] = 0

                temp = calculateObjective(NeighbourBoard, NeighbourState)

                if temp < opObjective:  # Found a better state
                    opObjective = temp
                    opState = NeighbourState.copy()

                # Reset back
                NeighbourBoard[j][i] = 0
                NeighbourState[i] = original_row
                NeighbourBoard[original_row][i] = 1

    return opState

def hillClimbing(board, state):
    step = 0
    while True:
        currentObjective = calculateObjective(board, state)
        newState = getNeighbour(board, state)

        if compareStates(state, newState):
            break

        state = newState
        generateBoard(board, state)

        step += 1
        print(f"Step {step}: Current State: {state} | Objective: {currentObjective}")
        printBoard(board)
        print()  # Blank line for readability

# Initialize state and board
state = [0] * N
board = [[0] * N for _ in range(N)]
configureFromInput(board, state)
hillClimbing(board, state)

print("Final board configuration:")
printBoard(board)

Enter the size of the board (N): 4
Enter the initial positions of the queens (0 to N-1 for each column):
Position for queen in column 0 (row 0 to 3): 0
Position for queen in column 1 (row 0 to 3): 0
Position for queen in column 2 (row 0 to 3): 0
Position for queen in column 3 (row 0 to 3): 0
Step 1: Current State: [0, 3, 0, 0] | Objective: 6
1 0 1 1
0 0 0 0
0 0 0 0
0 1 0 0

Step 2: Current State: [1, 3, 0, 0] | Objective: 3
0 0 1 1
1 0 0 0
0 0 0 0
0 1 0 0

Step 3: Current State: [1, 3, 0, 2] | Objective: 1
0 0 1 0
1 0 0 0
0 0 0 1
0 1 0 0

Final board configuration:
0 0 1 0
1 0 0 0
0 0 0 1
0 1 0 0


In [21]:
# moving rows method
# it evaluates all neighbors generated through row moves and selects the neighbor with the lowest cost.
# It provides a summary of the best move selected in each step and shows the current board configuration clearly.
# (focus is on moving queens in a more constrained manner, allowing gradual exploration.)
import random

class NQueens:
    def __init__(self, n, initial_board=None):
        self.n = n
        if initial_board is None:
            self.board = self.random_board()
        else:
            self.board = initial_board
        self.heuristic = self.calculate_heuristic(self.board)

    def random_board(self):
        """Create a random initial board configuration in 1-based indexing."""
        return [random.randint(1, self.n) for _ in range(self.n)]

    def calculate_heuristic(self, board):
        """Calculate the heuristic based on the number of pairs of queens attacking each other."""
        attacks = 0
        for i in range(self.n):
            for j in range(i + 1, self.n):
                if board[i] == board[j] or abs(board[i] - board[j]) == abs(i - j):
                    attacks += 1
        return attacks

    def get_neighbors(self, board):
        """Generate all neighboring states by moving each queen to a different row in its column."""
        neighbors = []
        for col in range(self.n):
            for row in range(1, self.n + 1):  # Maintain 1-based indexing
                if row != board[col]:  # Don't move the queen to the same row
                    new_board = board[:]
                    new_board[col] = row
                    neighbors.append(new_board)
        return neighbors

    def display_board(self, board):
        """Display the board in a grid format."""
        for row in range(1, self.n + 1):
            line = ""
            for col in range(self.n):
                if board[col] == row:
                    line += "Q "
                else:
                    line += ". "
            print(line)
        print()  # Extra newline for better separation

    def hill_climbing(self):
        current_board = self.board
        current_heuristic = self.heuristic

        while current_heuristic > 0:
            print(f"Current board (Heuristic: {current_heuristic}):")
            self.display_board(current_board)
            neighbors = self.get_neighbors(current_board)
            next_board = None
            next_heuristic = current_heuristic

            for neighbor in neighbors:
                neighbor_heuristic = self.calculate_heuristic(neighbor)
                print(f"Neighbor: {neighbor} with heuristic: {neighbor_heuristic}")

                if neighbor_heuristic < next_heuristic:
                    next_board = neighbor
                    next_heuristic = neighbor_heuristic

            if next_board is None:  # No better neighbors found
                print("No better neighbor found. Stopping search.")
                break

            current_board = next_board
            current_heuristic = next_heuristic

        print(f"Final board (Heuristic: {current_heuristic}):")
        self.display_board(current_board)
        if current_heuristic == 0:
            print("Solution found!")
        else:
            print("No solution found.")

    @staticmethod
    def validate_board(board, n):
        """Validate if the input board configuration is valid for N-Queens."""
        if len(board) != n:
            return False
        for pos in board:
            if pos < 1 or pos > n:  # Check for 1-based indexing
                return False
        return True

if __name__ == "__main__":
    n = int(input("Enter the number of queens (N): "))
    if n < 4:
        print("N must be at least 4 for a valid N-Queens problem.")
    else:
        user_input = input("Enter the initial board configuration (e.g., 1 2 3 4 for 4 queens): ")
        initial_board = list(map(int, user_input.split()))

        if NQueens.validate_board(initial_board, n):
            solver = NQueens(n, initial_board)
            solver.hill_climbing()
        else:
            print("Invalid board configuration. Please ensure it contains N integers between 1 and N.")


Enter the number of queens (N): 4
Enter the initial board configuration (e.g., 1 2 3 4 for 4 queens): 4 2 3 1
Current board (Heuristic: 2):
. . . Q 
. Q . . 
. . Q . 
Q . . . 

Neighbor: [1, 2, 3, 1] with heuristic: 4
Neighbor: [2, 2, 3, 1] with heuristic: 2
Neighbor: [3, 2, 3, 1] with heuristic: 3
Neighbor: [4, 1, 3, 1] with heuristic: 2
Neighbor: [4, 3, 3, 1] with heuristic: 4
Neighbor: [4, 4, 3, 1] with heuristic: 3
Neighbor: [4, 2, 1, 1] with heuristic: 3
Neighbor: [4, 2, 2, 1] with heuristic: 4
Neighbor: [4, 2, 4, 1] with heuristic: 2
Neighbor: [4, 2, 3, 2] with heuristic: 3
Neighbor: [4, 2, 3, 3] with heuristic: 2
Neighbor: [4, 2, 3, 4] with heuristic: 4
No better neighbor found. Stopping search.
Final board (Heuristic: 2):
. . . Q 
. Q . . 
. . Q . 
Q . . . 

No solution found.
